# Train a JointVAE model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import torch
from viz.visualize import Visualizer
use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change to your device

#### Prepare data list

In [4]:
!ls data

dress_dresslen_train_test_splits.json	dress_sleeve_train_test_splits.json
dress_sleevelen_train_test_splits.json	loadable_women_primary_dress.csv


#### Create list of image paths

In [5]:
loadable_dresses = list(np.loadtxt('data/loadable_women_primary_dress.csv',delimiter=',',skiprows=1,dtype='str'))

In [6]:
print(len(loadable_dresses))
print(loadable_dresses)[:5])

['/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/4/6418008_9882769.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/6/6627534_9864695.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/7/6772508_9949243.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/7/6758001_9588597.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/6/6637527_9387666.jpg']

In [ ]:
bad_data = ['/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg']
for i, bad in enumerate(bad_data):
    if bad in loadable_dresses:
        del loadable_dresses[i]
        
print(len(loadable_dresses))

#### Split into train and test set

In [7]:
#image_paths_train = loadable_dresses[:int(len(loadable_dresses)*0.87)]
#image_paths_test = loadable_dresses[int(len(loadable_dresses)*0.87):]
image_paths_train = loadable_dresses[:88800]
image_paths_test = loadable_dresses[88800:-125]


print(f"Number of train image paths: {len(image_paths_train):,d}")
print(f"Number of test image paths: {len(image_paths_test):,d}")
print()
print("Sample paths:")
print(image_paths_train[0])
print(image_paths_train[-1])
print(image_paths_test[0])
print(image_paths_test[-1])

Number of train image paths: 88,800
Number of test image paths: 13,200

Sample paths:
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/4/6418008_9882769.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/5/0/505051_864215.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/5/2/529420_884192.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/3/4/341494_659926.jpg


In [8]:
#from utils.dataloaders import get_mnist_dataloaders, get_fashion_mnist_dataloaders
#train_loader, test_loader = get_mnist_dataloaders(batch_size=64)
#train_loader, test_loader = get_fashion_mnist_dataloaders(batch_size=64)

In [9]:
from torchvision import transforms
#from utils.dataloaders_custom import get_imagelist_dataloader, ImageListDataset
from utils.dataloader_tools import get_imagelist_dataloader, ImageListDataset

BATCH_SIZE = 200

composed = transforms.Compose([transforms.CenterCrop((90,90)),transforms.Resize((64,64)),transforms.ToTensor()])

train_dataset = ImageListDataset(image_paths_train, cut_from='top', cut_amount=90, transform=composed)
test_dataset = ImageListDataset(image_paths_test, cut_from='top', cut_amount=90, transform=composed)

train_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=train_dataset)
test_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=test_dataset)

### Define latent distribution of the model

In [10]:
# Latent distribution will be joint distribution of 10 gaussian normal distributions
# and one 10 dimensional Gumbel Softmax distribution
latent_spec = {'cont': 20,
               'disc': [20]}

### Build a model

In [11]:
#from jointvae.models_v1 import VAE
from jointvae.models import VAE

#model = VAE(latent_spec=latent_spec, img_size=(3, 260, 260), use_cuda=use_cuda)
model = VAE(latent_spec=latent_spec, img_size=(3, 64, 64), use_cuda=use_cuda)

In [12]:
#print(model)

### Train the model

In [13]:
from torch import optim

# Build optimizer
optimizer = optim.Adam(model.parameters(), lr=3e-5, amsgrad=True) # added amsgrad # orig lr 5e-4

In [14]:
from jointvae.training import Trainer

# Define the capacities
# Continuous channels
cont_capacity = [0.0, 5.0, 25000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0
# Discrete channels
disc_capacity = [0.0, 5.0, 25000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0

# Build a trainer
trainer = Trainer(model, optimizer,
                  cont_capacity=cont_capacity,
                  disc_capacity=disc_capacity,
                 use_cuda=use_cuda)

#### Initialize visualizer

In [15]:
# Build a visualizer which will be passed to trainer to visualize progress during training
viz = Visualizer(model)

In [ ]:
# Train model for 10 epochs
# Note this should really be a 100 epochs and trained on a GPU, but this is just to demo

trainer.train(train_loader, epochs=130, save_training_gif=None)

0/88800	Loss: 2839.968
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
10000/88800	Loss: 2838.217
20000/88800	Loss: 2834.502
30000/88800	Loss: 2815.411
40000/88800	Loss: 2680.577
50000/88800	Loss: 2513.700
60000/88800	Loss: 2376.255
70000/88800	Loss: 2287.066
80000/88800	Loss: 2244.705
Epoch: 1 Average loss: 2540.00
0/88800	Loss: 2177.131
10000/88800	Loss: 2194.596
20000/88800	Loss: 2175.774
30000/88800	Loss: 2147.385
40000/88800	Loss: 2124.159
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
50000/88800	Loss: 2117.929
60000/88800	Loss: 2108.421
70000/88800	Loss: 2100.361
80000/88800	Loss: 2104.926
Epoch: 2 Average loss: 2130.48
0/88800	Loss: 2086.430
10000/88800	Loss: 2093.809
20000/88800	Loss: 2090.774
30000/88800	Loss: 2079.640
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all

0/88800	Loss: 1833.526
10000/88800	Loss: 1822.111
20000/88800	Loss: 1826.019
30000/88800	Loss: 1815.649
40000/88800	Loss: 1821.718
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
50000/88800	Loss: 1815.659
60000/88800	Loss: 1821.437
70000/88800	Loss: 1818.920
80000/88800	Loss: 1817.899
Epoch: 21 Average loss: 1819.60
0/88800	Loss: 1807.949
10000/88800	Loss: 1816.990
20000/88800	Loss: 1811.116
30000/88800	Loss: 1819.098
40000/88800	Loss: 1818.519
50000/88800	Loss: 1818.200
60000/88800	Loss: 1814.811
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
70000/88800	Loss: 1815.288
80000/88800	Loss: 1808.034
Epoch: 22 Average loss: 1814.45
0/88800	Loss: 1770.191
10000/88800	Loss: 1809.022
20000/88800	Loss: 1813.251
30000/88800	Loss: 1813.718
40000/88800	Loss: 1810.882
dimensions error in path: /home/jovyan/vmldata/

0/88800	Loss: 1791.135
10000/88800	Loss: 1752.929
20000/88800	Loss: 1759.270
30000/88800	Loss: 1757.794
40000/88800	Loss: 1753.608
50000/88800	Loss: 1756.008
60000/88800	Loss: 1752.154
70000/88800	Loss: 1758.734
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
80000/88800	Loss: 1751.966
Epoch: 41 Average loss: 1755.26
0/88800	Loss: 1717.332
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
10000/88800	Loss: 1754.405
20000/88800	Loss: 1751.838
30000/88800	Loss: 1754.640
40000/88800	Loss: 1747.375
50000/88800	Loss: 1752.210
60000/88800	Loss: 1749.860
70000/88800	Loss: 1755.368
80000/88800	Loss: 1751.352
Epoch: 42 Average loss: 1752.22
0/88800	Loss: 1786.812
10000/88800	Loss: 1745.424
20000/88800	Loss: 1754.923
30000/88800	Loss: 1752.329
40000/88800	Loss: 1753.314
50000/88800	Loss: 1743.701
60000/88800	Loss: 17

0/88800	Loss: 1742.285
10000/88800	Loss: 1723.906
20000/88800	Loss: 1721.341
30000/88800	Loss: 1716.810
40000/88800	Loss: 1719.586
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
50000/88800	Loss: 1712.264
60000/88800	Loss: 1725.588
70000/88800	Loss: 1721.281
80000/88800	Loss: 1718.276
Epoch: 61 Average loss: 1720.22
0/88800	Loss: 1706.503
10000/88800	Loss: 1718.906
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
20000/88800	Loss: 1717.461
30000/88800	Loss: 1718.213
40000/88800	Loss: 1717.892
50000/88800	Loss: 1717.431
60000/88800	Loss: 1718.711
70000/88800	Loss: 1719.919
80000/88800	Loss: 1717.495
Epoch: 62 Average loss: 1718.77
0/88800	Loss: 1688.114
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
10000/88800	Los

0/88800	Loss: 1719.855
10000/88800	Loss: 1720.282
20000/88800	Loss: 1709.799
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
30000/88800	Loss: 1716.044
40000/88800	Loss: 1709.119
50000/88800	Loss: 1707.216
60000/88800	Loss: 1713.463
70000/88800	Loss: 1707.106
80000/88800	Loss: 1705.289
Epoch: 81 Average loss: 1710.75
0/88800	Loss: 1713.395
dimensions error in path: /home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg
replacing with previous image
10000/88800	Loss: 1710.502
20000/88800	Loss: 1707.359
30000/88800	Loss: 1711.536
40000/88800	Loss: 1710.821
50000/88800	Loss: 1715.082
60000/88800	Loss: 1703.256
70000/88800	Loss: 1716.270
80000/88800	Loss: 1709.981
Epoch: 82 Average loss: 1710.64
0/88800	Loss: 1729.920


In [ ]:
print('hi')

### Visualize

In [ ]:
# Plot reconstructions
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Get a batch of data
for batch, labels in test_loader:
    break

# Reconstruct data using Joint-VAE model
recon = viz.reconstructions(batch)

plt.figure(figsize=(26,26))
#np.transpose(recon.numpy(), (2,1,0))
plt.imshow(np.rot90(np.transpose(recon.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_reconstructions.png",dpi=200)

In [ ]:
# Plot samples
samples = viz.samples()

plt.figure(figsize=(26,26))
#plt.imshow(samples.numpy()[0, :, :],cmap='gray');
plt.imshow(np.rot90(np.transpose(samples.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30.png",dpi=200)

#### Traverses all latent dimensions one by one and plots a grid of images where each row corresponds to a latent traversal of one latent dimension

In [ ]:
# Plot all traversals
traversals = viz.all_latent_traversals(size=20)

plt.figure(figsize=(26,26))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_all_traversals_n20.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=2, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30.png_traversals2100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=1, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
plt.imshow(traversals.numpy()[0, :, :]);
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_traversals1100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=9, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
#plt.imshow(traversals.numpy()[0, :, :], cmap='gray');
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_traversals9100.png",dpi=200)

In [ ]:
!ls

### Save Model

In [28]:
model_name = "croptotop_64x64_e130_b200_c20d20_gam30.pth"

In [29]:
torch.save(model.state_dict(),"trained_models" + "statedict_" + model_name) # save state dict
torch.save(model, model_name) # save full model

In [30]:
print("Done training: ",model_name)

Done training:  realdata64x64_e60_b512_cd40_gam20.pth


#### Restore Model from State Dict

In [28]:
sd_model = VAE(latent_spec=latent_spec, img_size=(3, 64, 64))
sd_model.load_state_dict(torch.load("statedict_" + model_name))

#### Restore Full Model
* Note in this case the serialized data is bound to the specific classes and exact directory strucutre used.

In [ ]:
full_model = torch.load(model_name)

In [ ]:
type(full_model)

In [ ]:
type(sd_model)

In [36]:
!ls

data				requirements.txt
dataloading_pytorch_test.ipynb	statedict_jvae_fmnist_oct292018.pth
imgs				trained_models
jointvae			training.gif
jvae_fmnist_oct292018.pth	training_rd1.gif
latent_traversals.py		train_model.ipynb
load_model.ipynb		train_model_realdata_306_260_v1.ipynb
main.py				train_model_realdata_64_v1.ipynb
__pycache__			utils
RandomUtilsandTests.ipynb	viz
README.md
